# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [2]:
import jax.numpy as np
import jax
from jax import dlpack as jdlpack
from torch.utils import dlpack as tdlpack
import epcsaft
from importlib import reload
import ml_pc_saft
import torch
import test_epcsaft
import test_cython

from jax.config import config
config.update("jax_enable_x64", True)

In [ ]:
reload(epcsaft)

## Tests 

In [ ]:
reload(test_epcsaft)

In [ ]:
reload(test_cython)

In [ ]:
test_epcsaft.test_indexes(True)

In [ ]:
test_epcsaft.test_hres(True)

In [ ]:
test_epcsaft.test_gres(True)

In [ ]:
test_epcsaft.test_sres(True)

In [ ]:
test_epcsaft.test_pressure(True)

In [ ]:
test_epcsaft.test_density(True)

In [ ]:
test_epcsaft.test_VP(True)

## debug

In [40]:
x = np.asarray([0.3607, 0.6393])[..., np.newaxis]
m = np.asarray([2.5313, 2.3827])[..., np.newaxis]
s = np.asarray([3.4608, 3.1771])[..., np.newaxis]
e = np.asarray([269.47, 198.24])[..., np.newaxis]
vol_a = np.asarray([0.0, 0.032384])[..., np.newaxis]
e_assoc = np.asarray([0.0, 2653.4])[..., np.newaxis]
dipm = np.asarray([1.04, 0.])[..., np.newaxis]
dip_num = np.asarray([1.0, 0.])[..., np.newaxis]
k_ij = np.asarray([[0.0, 0.0],
                    [0.0, 0.0]])
z = np.zeros_like(m)
dielc = np.ones_like(m)
l_ij = khb_ij = np.zeros_like(k_ij)

ref = 101330.0 # source: Chen GH, Wang Q, Ma ZM, Yan XH, Han SJ. Phase equilibria at superatmospheric pressures for systems containing halohydrocarbon, aromatic hydrocarbon, and alcohol. Journal of Chemical and Engineering Data. 1995 Mar;40(2):361-6.
t = 337.03

In [41]:
kb = 1.380648465952442093e-23  # Boltzmann constant, J K^-1
N_AV = 6.022140857e23  # Avogadro's number

In [42]:
rho = epcsaft.pcsaft_den(x, m, s, e, t, 101325.0, k_ij, l_ij,
               khb_ij, e_assoc, vol_a, dipm,
               dip_num, z, dielc, 1.0)

In [56]:
epcsaft.pcsaft_gres(x, m, s, e, t, rho, k_ij, l_ij,
                khb_ij, e_assoc, vol_a, dipm,
                dip_num, z, dielc)/(kb*N_AV*t)

Array(-0.08985061, dtype=float64)

## ML layer

In [4]:
                            #m      s       e     vola      eassoc dipm dipnum z   dielc  k    l   khb
paratorch = torch.tensor([[2.5313, 3.4608, 269.47, 0.0,     0.0,    1.04, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                          [2.3827, 3.1771, 198.24, 0.032384, 2653.4, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]], dtype=float, device='cuda:0', requires_grad=True)


In [5]:
statetorch = torch.asarray([0.3607, 0.6393, 337.03, 101330.0, 1.0], dtype=float, device='cuda:0')


In [58]:
reload(ml_pc_saft)

<module 'ml_pc_saft' from '/home/wildsonbbl/documents/code/ePC-SAFT/ml_pc_saft.py'>

In [59]:
rho = ml_pc_saft.rho
ActivityCoefficient = ml_pc_saft.ActivityCoefficient

In [8]:
parameters = tdlpack.to_dlpack(paratorch)
parameters = jdlpack.from_dlpack(parameters)

state = tdlpack.to_dlpack(statetorch)
state = jdlpack.from_dlpack(state)

In [22]:
fakebatch_para = np.ones((62,1,1))*parameters
fakebatch_state = np.ones((62,1,))*state

In [60]:
rhon = rho(parameters, state)
rhon

Array(14205.53155798, dtype=float64)

In [61]:
ActivityCoefficient(parameters, state)

Array(1.88563324, dtype=float64)

In [62]:
gradAC = jax.jacfwd(ActivityCoefficient,0)

In [64]:
gradAC(parameters + 1.0e-3,state)

Array([[-2.66684705e-04, -4.08809933e-04, -1.11254787e-07,
        -9.37161077e-02, -2.82243544e-07,  0.00000000e+00,
         0.00000000e+00, -7.33380058e-07,  3.68295796e-10,
         0.00000000e+00,  2.81429756e-04,  3.76281583e-04],
       [ 1.64407024e-04,  2.24259396e-02,  1.19978811e-07,
         7.05222804e-01,  6.98762368e-05,  0.00000000e+00,
         0.00000000e+00,  2.93728906e-06, -1.46914956e-09,
         0.00000000e+00,  2.81429756e-04,  3.76281583e-04]],      dtype=float64)

In [79]:
class PCSAFT(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, state):
        parameters = tdlpack.to_dlpack(input)
        parameters = jdlpack.from_dlpack(parameters)

        state = tdlpack.to_dlpack(state)
        state = jdlpack.from_dlpack(state)

        ctx.parameters = parameters
        ctx.state = state
        gamma1 = ActivityCoefficient(parameters, state)
        gamma1 = jdlpack.to_dlpack(gamma1)

        gamma1 = tdlpack.from_dlpack(gamma1)
        return gamma1

    @staticmethod
    def backward(ctx, dg1):

        grad_gamma1 = gradAC(ctx.parameters + 1.0e-3, ctx.state)
        grad_gamma1 = jdlpack.to_dlpack(grad_gamma1)
        grad_gamma1 = dg1 * tdlpack.from_dlpack(grad_gamma1)
        return grad_gamma1, None


In [76]:
pcsaft = PCSAFT.apply

In [80]:
gamma = pcsaft(paratorch,statetorch)

In [81]:
gamma

tensor(1.8856, device='cuda:0', dtype=torch.float64, grad_fn=<PCSAFTBackward>)

In [83]:
gamma.backward()

tensor(1., device='cuda:0', dtype=torch.float64)


In [82]:
paratorch.grad

tensor([[-8.0005e-04, -1.2264e-03, -3.3376e-07, -2.8115e-01, -8.4673e-07,
          0.0000e+00,  0.0000e+00, -2.2001e-06,  1.1049e-09,  0.0000e+00,
          8.4429e-04,  1.1288e-03],
        [ 4.9322e-04,  6.7278e-02,  3.5994e-07,  2.1157e+00,  2.0963e-04,
          0.0000e+00,  0.0000e+00,  8.8119e-06, -4.4074e-09,  0.0000e+00,
          8.4429e-04,  1.1288e-03]], device='cuda:0', dtype=torch.float64)